# Instagram Post insights analysis

## This is an insight generator and prediction tool for your Instagram posts

### Steps for data extraction:
1. Download your Instagram analytic data by going to your Instagram account setting then go to > privacy and security option > Data Download > Request Download.
2. Create the folder "assets" in the root directory of the project.
3. Extract the "past_instagram_insights" folder from this downloaded dataset in step 1 to this "assets" folder from step 2
4. Copy and paste another folder "media" (which should be found in the downloaded dataset from step 2) to the path "assets/past_instagram_insights/"


TODO:

1. Analyze the comments
2. Analyze Videos
   1. Analyze audio from videos and Reels


## Define all the Global variables
Run this cell first

In [91]:
# All the global variables are defines in this first cell
!pip install pandas --upgrade
import pandas as pd

parent_folder_path = 'assets/past_instagram_insights/'
file_name = 'posts'


Defaulting to user installation because normal site-packages is not writeable


## Load the json data and convert it to a CSV


In [92]:
import json
import csv
from datetime import datetime

# [convert_json_to_csv] converts the given json file to csv


def convert_json_to_csv(parent_folder_path, file_name, top_json_key):

    # Opening JSON file and loading the data
    # into the variable data
    with open(parent_folder_path+file_name+'.json') as json_file:
        data = json.load(json_file)

    json_data = data[top_json_key]

    # now we will open a file for writing
    data_file = open(parent_folder_path+file_name+'.csv', 'w')

    # create the csv writer object
    csv_writer = csv.writer(data_file)

    # Counter variable used for writing
    # headers to the CSV file
    count = 0

    for data in json_data:
        if count == 0:
            # Writing headers of CSV file
            #header = list(data.keys())
            csv_writer.writerow(
                ['title', 'post_media', 'creation_timestamp', 'post_likes'])
            count += 1

        # Writing data of CSV fileu
        # cell = [data['media_map_data']['Media Thumbnail']['title'], path_to_image_html(parent_folder_path, data['media_map_data']['Media Thumbnail']['uri']),
        #         data['media_map_data']['Media Thumbnail']['creation_timestamp'], data['string_map_data']['Likes']['value']]
        cell = [data['media_map_data']['Media Thumbnail']['title'], data['media_map_data']['Media Thumbnail']['uri'],
                datetime.fromtimestamp(data['media_map_data']['Media Thumbnail']['creation_timestamp']), data['string_map_data']['Likes']['value']]
        csv_writer.writerow(cell)

    data_file.close()


## Call the convert_json_to_csv function to convert json to csv


In [93]:
# Convert the json into CSV
convert_json_to_csv(parent_folder_path, file_name, 'organic_insights_posts')

# Load this saved csv
csv = pd.read_csv(parent_folder_path + file_name+'.csv')


## Load this saved csv


In [94]:
import pandas as pd

# Load the saved csv
df = pd.read_csv(parent_folder_path + file_name+'.csv', index_col=False)

print('Shape: ', df.shape)
print('Describe: ', df.post_likes.describe())

# Limit the rows for testing
df = df.head(50)
df


Shape:  (540, 4)
Describe:  count    540.000000
mean      35.866667
std       46.979299
min        3.000000
25%       16.000000
50%       25.000000
75%       39.000000
max      591.000000
Name: post_likes, dtype: float64


,title,post_media,creation_timestamp,post_likes
0,Python â¢\nTag Your Programmer groups â¢\nâ...,media/posts/201912/69503764_1316105261901993_3...,2019-12-03 10:45:34,19
1,@multiverseapp Follow us @multiverseapp for mo...,media/posts/201912/79984281_434751277472386_56...,2019-12-02 12:03:00,19
2,Beautiful Onboarding animation concept of a 'I...,media/posts/201912/FQIoAkMzLBdAG3bItDlYEBgSZGF...,2019-12-02 12:00:26,30
3,@multiverseapp Follow @multiverseapp for daily...,media/posts/201912/79862581_805719879875319_32...,2019-12-02 11:58:03,19
4,@welovewebdesign by Miroslav SkoÄdopoly\nFoll...,media/posts/201912/78766756_198610067846038_40...,2019-12-02 11:34:20,18
5,Minimal Music Player App UI Design Concept | ð...,media/posts/201912/75439560_533100217244996_42...,2019-12-02 05:38:59,27
6,What's your opinion? UI design by @yili0909 .\...,media/posts/201912/73420512_531336587710615_49...,2019-12-02 05:37:04,11
7,Web Concept \nFollowÂ @multiverseapp for more!...,media/posts/201912/72617744_172397430627510_75...,2019-12-02 01:29:12,16
8,"11 years ago the MCU started with him, which h...",media/posts/201912/78841972_515651432638123_58...,2019-12-02 01:16:43,38
9,Official app of MCU? Yay or nay?\n_ \n#mobilea...,media/posts/201912/74655045_443186606597342_57...,2019-12-01 22:57:01,144


[38815:38839:1227/205906.327145:ERROR:get_updates_processor.cc(257)] PostClientToServerMessage() failed during GetUpdates with error 4


## Encode the post likes

In [95]:
# Get the 25th percentile, 50th percentile and 75th percentile of post_likes
# We will use these as the anchors to classify posts if they are of low, medium or high engagement
low_like_count = df.post_likes.describe()['25%']
medium_like_count = df.post_likes.describe()['50%']
high_like_count = df.post_likes.describe()['75%']

# Loop through all the post likes and encode them to low, medium, and high likes
for post_likes_index in range(len(df.post_likes)):
    # Like count of the current post iteration
    post_like = df.post_likes[post_likes_index]

    if post_like <= low_like_count:
        df.post_likes[post_likes_index] = 'low'
    elif post_like > low_like_count and post_like <= medium_like_count:
        df.post_likes[post_likes_index] = 'medium'
    else:
        df.post_likes[post_likes_index] = 'high'

df.post_likes

/home/praharshb/.local/lib/python3.10/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


0     medium
1     medium
2       high
3     medium
4     medium
5       high
6        low
7        low
8       high
9       high
10      high
11    medium
12    medium
13      high
14      high
15      high
16      high
17      high
18       low
19    medium
20      high
21    medium
22    medium
23      high
24      high
25    medium
26       low
27       low
28       low
29    medium
30       low
31       low
32       low
33       low
34      high
35      high
36       low
37      high
38       low
39      high
40      high
41      high
42       low
43      high
44      high
45      high
46      high
47      high
48    medium
49      high
Name: post_likes, dtype: object

## Fetch hashtags from the title


In [96]:
titles = df.title

all_hashtags = []

# Go through the title, and separate out all the hashtags
for i in range(len(titles)):
    # Define an empty hashtag list
    # If no hashtag is found, this empty list is returned
    hashtags_column = []
    title = titles[i]

    if (isinstance(title, str) and '#' in title):
        first_hashtag = title.find('#')
        title = title[first_hashtag:]
        hashtags_column = title.split('#')
        hashtags_column.remove('')

        # remove blank spaces
        for j in range(len(hashtags_column)):
            hashtags_column[j] = hashtags_column[j].replace(' ', '')

    all_hashtags.append(hashtags_column)

# Go through this column to get the most occuring hashtags
hashtags_dict = {}
for hashtag_cell in all_hashtags:
    for single_hashtag in hashtag_cell:
        if single_hashtag not in hashtags_dict:
            hashtags_dict[single_hashtag] = 1
        else:
            hashtags_dict[single_hashtag] += 1

# Rearrange the dict to have the highest occurrence first
hashtags_dict = {k: v for k, v in sorted(
    hashtags_dict.items(), key=lambda item: item[1], reverse=True)}

# Get the first n hashtags from the most used hasttags
max_hashtag_count = 5
most_occuring_hashtags = list(hashtags_dict.keys())[:max_hashtag_count]
print('Most occuring hashtags', most_occuring_hashtags)

# Go through hashtags from the dataframe and set the
hashtag_distribution = []
hashtag_distribution

# Add these hashtags as new columns
# add appropriate values for the posts in which these hashtags occur
for row_hashtags in all_hashtags:
    hashtag_distribution_row = []
    for most_occuring_hashtag in most_occuring_hashtags:
        if most_occuring_hashtag in row_hashtags:
            hashtag_distribution_row.append(1)
        else:
            hashtag_distribution_row.append(0)
    hashtag_distribution.append(hashtag_distribution_row)

# Add this occurrence frequency to the dataframe
for most_occuring_hashtag in most_occuring_hashtags:
    df[most_occuring_hashtags] = hashtag_distribution

df


Most occuring hashtags ['appdesign', 'dailyui', 'mobiledesign', 'uitrends', 'userinterfacedesign']


,title,post_media,creation_timestamp,post_likes,appdesign,dailyui,mobiledesign,uitrends,userinterfacedesign
0,Python â¢\nTag Your Programmer groups â¢\nâ...,media/posts/201912/69503764_1316105261901993_3...,2019-12-03 10:45:34,medium,0,0,0,0,0
1,@multiverseapp Follow us @multiverseapp for mo...,media/posts/201912/79984281_434751277472386_56...,2019-12-02 12:03:00,medium,1,0,1,1,1
2,Beautiful Onboarding animation concept of a 'I...,media/posts/201912/FQIoAkMzLBdAG3bItDlYEBgSZGF...,2019-12-02 12:00:26,high,0,0,0,0,0
3,@multiverseapp Follow @multiverseapp for daily...,media/posts/201912/79862581_805719879875319_32...,2019-12-02 11:58:03,medium,0,0,0,0,0
4,@welovewebdesign by Miroslav SkoÄdopoly\nFoll...,media/posts/201912/78766756_198610067846038_40...,2019-12-02 11:34:20,medium,0,0,0,0,0
5,Minimal Music Player App UI Design Concept | ð...,media/posts/201912/75439560_533100217244996_42...,2019-12-02 05:38:59,high,1,1,0,0,0
6,What's your opinion? UI design by @yili0909 .\...,media/posts/201912/73420512_531336587710615_49...,2019-12-02 05:37:04,low,1,0,0,1,0
7,Web Concept \nFollowÂ @multiverseapp for more!...,media/posts/201912/72617744_172397430627510_75...,2019-12-02 01:29:12,low,0,0,0,0,0
8,"11 years ago the MCU started with him, which h...",media/posts/201912/78841972_515651432638123_58...,2019-12-02 01:16:43,high,0,1,0,1,1
9,Official app of MCU? Yay or nay?\n_ \n#mobilea...,media/posts/201912/74655045_443186606597342_57...,2019-12-01 22:57:01,high,1,1,1,1,1


## Saperate all the date fields


In [99]:
dates = df.creation_timestamp

weekdays = []
times = []

for i in range(len(dates)):
    date = datetime.fromisoformat(dates[i])
    # Add day of the week as an integer, where Monday is 0 and Sunday is 6.
    weekdays.append(date.weekday())

    # Add time as a float
    times.append(date.time().hour + (date.time().minute / 60))

df['post_weekday'] = weekdays
df['post_time'] = times

df


,title,post_media,creation_timestamp,post_likes,appdesign,dailyui,mobiledesign,uitrends,userinterfacedesign,post_weekday,post_time
0,Python â¢\nTag Your Programmer groups â¢\nâ...,media/posts/201912/69503764_1316105261901993_3...,2019-12-03 10:45:34,medium,0,0,0,0,0,1,10.750000
1,@multiverseapp Follow us @multiverseapp for mo...,media/posts/201912/79984281_434751277472386_56...,2019-12-02 12:03:00,medium,1,0,1,1,1,0,12.050000
2,Beautiful Onboarding animation concept of a 'I...,media/posts/201912/FQIoAkMzLBdAG3bItDlYEBgSZGF...,2019-12-02 12:00:26,high,0,0,0,0,0,0,12.000000
3,@multiverseapp Follow @multiverseapp for daily...,media/posts/201912/79862581_805719879875319_32...,2019-12-02 11:58:03,medium,0,0,0,0,0,0,11.966667
4,@welovewebdesign by Miroslav SkoÄdopoly\nFoll...,media/posts/201912/78766756_198610067846038_40...,2019-12-02 11:34:20,medium,0,0,0,0,0,0,11.566667
5,Minimal Music Player App UI Design Concept | ð...,media/posts/201912/75439560_533100217244996_42...,2019-12-02 05:38:59,high,1,1,0,0,0,0,5.633333
6,What's your opinion? UI design by @yili0909 .\...,media/posts/201912/73420512_531336587710615_49...,2019-12-02 05:37:04,low,1,0,0,1,0,0,5.616667
7,Web Concept \nFollowÂ @multiverseapp for more!...,media/posts/201912/72617744_172397430627510_75...,2019-12-02 01:29:12,low,0,0,0,0,0,0,1.483333
8,"11 years ago the MCU started with him, which h...",media/posts/201912/78841972_515651432638123_58...,2019-12-02 01:16:43,high,0,1,0,1,1,0,1.266667
9,Official app of MCU? Yay or nay?\n_ \n#mobilea...,media/posts/201912/74655045_443186606597342_57...,2019-12-01 22:57:01,high,1,1,1,1,1,6,22.950000


## Get Dominant color from an image


In [100]:
!pip install webcolors --upgrade
!pip install colorthief --upgrade
from colorthief import ColorThief
import webcolors

# To find the closest colour name https://stackoverflow.com/a/9694246/6559381


def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]


post_media = df.post_media

dominant_colors = []

for i in range(len(post_media)):
    # Get the image from the list of images
    image = post_media[i]

    # Get the dominant color
    if '.jpg' in image:
        print('Finding the dominant color in the image: ', i)
        color_thief = ColorThief(parent_folder_path+image)
        dominant_color = color_thief.get_color(quality=1)
        dominant_colors.append(closest_colour((dominant_color)))
    else:
        print(i, ' is a video')
        dominant_colors.append('video')

df['post_dominant_colors'] = dominant_colors
df


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Finding the dominant color in the image:  0
1  is a video
2  is a video
3  is a video
Finding the dominant color in the image:  4
Finding the dominant color in the image:  5
Finding the dominant color in the image:  6
Finding the dominant color in the image:  7
8  is a video
Finding the dominant color in the image:  9
10  is a video
11  is a video
12  is a video
Finding the dominant color in the image:  13
Finding the dominant color in the image:  14
Finding the dominant color in the image:  15
16  is a video
Finding the dominant color in the image:  17
Finding the dominant color in the image:  18
Finding the dominant color in the image:  19
Finding the dominant color in the image:  20
21  is a video
22  is a video
23  is a video
24  is a video
Finding the dominant color in the image:  25
Finding the dominant color in the image:  26

,title,post_media,creation_timestamp,post_likes,appdesign,dailyui,mobiledesign,uitrends,userinterfacedesign,post_weekday,post_time,post_dominant_colors
0,Python â¢\nTag Your Programmer groups â¢\nâ...,media/posts/201912/69503764_1316105261901993_3...,2019-12-03 10:45:34,medium,0,0,0,0,0,1,10.750000,black
1,@multiverseapp Follow us @multiverseapp for mo...,media/posts/201912/79984281_434751277472386_56...,2019-12-02 12:03:00,medium,1,0,1,1,1,0,12.050000,video
2,Beautiful Onboarding animation concept of a 'I...,media/posts/201912/FQIoAkMzLBdAG3bItDlYEBgSZGF...,2019-12-02 12:00:26,high,0,0,0,0,0,0,12.000000,video
3,@multiverseapp Follow @multiverseapp for daily...,media/posts/201912/79862581_805719879875319_32...,2019-12-02 11:58:03,medium,0,0,0,0,0,0,11.966667,video
4,@welovewebdesign by Miroslav SkoÄdopoly\nFoll...,media/posts/201912/78766756_198610067846038_40...,2019-12-02 11:34:20,medium,0,0,0,0,0,0,11.566667,darkslategray
5,Minimal Music Player App UI Design Concept | ð...,media/posts/201912/75439560_533100217244996_42...,2019-12-02 05:38:59,high,1,1,0,0,0,0,5.633333,darkslategray
6,What's your opinion? UI design by @yili0909 .\...,media/posts/201912/73420512_531336587710615_49...,2019-12-02 05:37:04,low,1,0,0,1,0,0,5.616667,lightsteelblue
7,Web Concept \nFollowÂ @multiverseapp for more!...,media/posts/201912/72617744_172397430627510_75...,2019-12-02 01:29:12,low,0,0,0,0,0,0,1.483333,darkslategray
8,"11 years ago the MCU started with him, which h...",media/posts/201912/78841972_515651432638123_58...,2019-12-02 01:16:43,high,0,1,0,1,1,0,1.266667,video
9,Official app of MCU? Yay or nay?\n_ \n#mobilea...,media/posts/201912/74655045_443186606597342_57...,2019-12-01 22:57:01,high,1,1,1,1,1,6,22.950000,midnightblue


## Drop unwanted columns


In [101]:
# Reset the index, make factor a column

if df.__contains__('title'):
    df.drop(columns=['title'], inplace=True)
if df.__contains__('creation_timestamp'):
    df.drop(columns=['creation_timestamp'], inplace=True)
if df.__contains__('post_media'):
    df.drop(columns=['post_media'], inplace=True)
if df.__contains__('hashtags'):
    df.drop(columns=['hashtags'], inplace=True)
df


,post_likes,appdesign,dailyui,mobiledesign,uitrends,userinterfacedesign,post_weekday,post_time,post_dominant_colors
0,medium,0,0,0,0,0,1,10.750000,black
1,medium,1,0,1,1,1,0,12.050000,video
2,high,0,0,0,0,0,0,12.000000,video
3,medium,0,0,0,0,0,0,11.966667,video
4,medium,0,0,0,0,0,0,11.566667,darkslategray
5,high,1,1,0,0,0,0,5.633333,darkslategray
6,low,1,0,0,1,0,0,5.616667,lightsteelblue
7,low,0,0,0,0,0,0,1.483333,darkslategray
8,high,0,1,0,1,1,0,1.266667,video
9,high,1,1,1,1,1,6,22.950000,midnightblue


## Save this clean Dataset
Since the dataset can be super long to compute expecially if there are many image posts, it is a good idea to save the cleaned version of the dataset after doing an EDA. This way, for any changes that are needed to be done in code in the future, this cleaned file can directly be loaded up without having to recompute the cleaned version again.

In [102]:
df.to_csv(parent_folder_path+file_name+'_clean.csv', index=False)


[38815:38839:1227/210906.590855:ERROR:get_updates_processor.cc(257)] PostClientToServerMessage() failed during GetUpdates with error 4


In [103]:
# Checkpoint- load the cleaned csv
import os

if os.path.isfile(parent_folder_path+file_name+'_clean.csv'):
    df = pd.read_csv(parent_folder_path+file_name+'_clean.csv')
    if (df.__contains__('unnamed 0')):
        df.drop(['unnamed 0'], axis=1)

df


,post_likes,appdesign,dailyui,mobiledesign,uitrends,userinterfacedesign,post_weekday,post_time,post_dominant_colors
0,medium,0,0,0,0,0,1,10.750000,black
1,medium,1,0,1,1,1,0,12.050000,video
2,high,0,0,0,0,0,0,12.000000,video
3,medium,0,0,0,0,0,0,11.966667,video
4,medium,0,0,0,0,0,0,11.566667,darkslategray
5,high,1,1,0,0,0,0,5.633333,darkslategray
6,low,1,0,0,1,0,0,5.616667,lightsteelblue
7,low,0,0,0,0,0,0,1.483333,darkslategray
8,high,0,1,0,1,1,0,1.266667,video
9,high,1,1,1,1,1,6,22.950000,midnightblue


## Visualise the most liked Data

From the entire dataset, show the charecteristics of the top n most liked posts

we will use _sweetviz_ to visualize the data


In [104]:
!pip install sweetviz --upgrade
import sweetviz as sv

# Creates the graph of the top n (index_to_truncate_from) most liked posts
def visualize_top_n_most(index_to_truncate_from):
    df_report = df.sort_values(
        by='post_likes', ascending=False).head(index_to_truncate_from)
    sv.analyze(df_report).show_html()


# Creates the report graph of the top 10 most liked posts
visualize_top_n_most(10)


Defaulting to user installation because normal site-packages is not writeable


Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.



[38815:38815:1227/211013.134920:ERROR:CONSOLE(1)] "[Shields]: Can't request shields panel data. Error: No tab url specified", source: chrome-extension://mnojpmjdmbbfmejpflffifhffcmidifd/out/brave_extension_background.bundle.js (1)
[38815:38815:1227/211013.135648:ERROR:CONSOLE(1)] "[Shields]: Can't request shields panel data. Error: No tab url specified", source: chrome-extension://mnojpmjdmbbfmejpflffifhffcmidifd/out/brave_extension_background.bundle.js (1)
[45302:45302:1227/211013.171390:ERROR:vaapi_wrapper.cc(1073)] vaQuerySurfaceAttributes failed, VA error: invalid parameter
[45302:45302:1227/211013.171618:ERROR:vaapi_wrapper.cc(1020)] FillProfileInfo_Locked failed for va_profile VAProfileH264Main and entrypoint VAEntrypointVLD
[45302:45302:1227/211013.171677:ERROR:vaapi_wrapper.cc(1073)] vaQuerySurfaceAttributes failed, VA error: invalid parameter
[45302:45302:1227/211013.171707:ERROR:vaapi_wrapper.cc(1020)] FillProfileInfo_Locked failed for va_profile VAProfileH264High and entryp

Opening in existing browser session.


## Encode categorical data


In [105]:
!pip install sklearn --upgrade
from sklearn.preprocessing import LabelEncoder

le_post_dominant_colors = LabelEncoder()
df['post_dominant_colors'] = le_post_dominant_colors.fit_transform(
    df['post_dominant_colors']
)

le_post_likes = LabelEncoder()
df['post_likes'] = le_post_likes.fit_transform(
    df['post_likes']
)19

df


Defaulting to user installation because normal site-packages is not writeable


,post_likes,appdesign,dailyui,mobiledesign,uitrends,userinterfacedesign,post_weekday,post_time,post_dominant_colors
0,2,0,0,0,0,0,1,10.750000,0
1,2,1,0,1,1,1,0,12.050000,11
2,0,0,0,0,0,0,0,12.000000,11
3,2,0,0,0,0,0,0,11.966667,11
4,2,0,0,0,0,0,0,11.566667,2
5,0,1,1,0,0,0,0,5.633333,2
6,1,1,0,0,1,0,0,5.616667,5
7,1,0,0,0,0,0,0,1.483333,2
8,0,0,1,0,1,1,0,1.266667,11
9,0,1,1,1,1,1,6,22.950000,7


## Create Training data sets


In [106]:
from sklearn.model_selection import train_test_split

# Let's extract features data assigning to X and labels data assigning to:
X = df.drop(['post_likes'], axis=1)
y = df['post_likes']

(X_train, X_tmp, y_train, y_tmp) = train_test_split(
    X, y, train_size=0.7, random_state=1)
(X_test, X_val, y_test, y_val) = train_test_split(
    X_tmp, y_tmp, train_size=0.6, random_state=1)


print('Training data shape: ', X_train.shape)
print('Validation data shape: ', X_val.shape)
print('Testing data shape: ', X_test.shape)


Training data shape:  (35, 8)
Validation data shape:  (6, 8)
Testing data shape:  (9, 8)


## Train Decision Tree


In [107]:
from IPython.display import Image
from subprocess import call
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier

# Defining and fitting a DecisionTreeClassifier instance
tree1 = DecisionTreeClassifier()
tree1.fit(X_train, y_train)


DecisionTreeClassifier()

## Validate the Decision tee


In [108]:
tree1.score(X_val, y_val)


0.5

## Test the Decision tee


In [109]:
tree1.score(X_test, y_test)


0.5555555555555556

## Make a Prediction

Now that we know how our decision tree works, let us make predictions.
This sample prediction should output how many likes will the post will receive with the given constraints


In [112]:
ui_trends = 1
appdesign = 1
mobile_design = 1
daily_ui = 0
user_interface_design = 1
post_weekday = 1
post_time = 14
post_dominant_colors = 7

sample_one_pred_tree1 = int(
    tree1.predict(
        [
            [
                ui_trends,	appdesign,	mobile_design,	daily_ui,	user_interface_design,	post_weekday,	post_time, post_dominant_colors
            ]
        ]
    )
)
sample_one_pred_tree1


/home/praharshb/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


1

In [113]:
if sample_one_pred_tree1 == 0:
    print(" This post will receive less likes.")
elif sample_one_pred_tree1 == 1:
    print(" This post will receive medium likes.")
else:
    print(" This post will receive high likes.")


 This post will receive medium likes.
